## **Structured Transformation of Retail Transaction Records**

### **Full Data**

#### 🧹✨ **1. Cleaning**

In [1076]:
# Importing the pandas library and giving it an alias 'pd' for data manipulation and analysis
import pandas as pd

In [1077]:
# Reading the CSV file named 'raw_data.csv' into a DataFrame called df_full
df_full = pd.read_csv("raw_data.csv")

#### *Handling Duplicated Values*

##### Before

In [1078]:
df_full.duplicated().sum()

1

##### Transformation

In [1079]:
# Drop duplicate rows from the dataset
df_full = df_full.drop_duplicates()

##### After

In [1080]:
df_full.duplicated().sum()

0

___
##### Transformation : Explanation
There was **1 duplicate row** in the dataset, which has been **removed** / **dropped**  to maintain data integrity and prevent **inaccurate analysis or double-counting** during aggregation or visualization.
___

#### *Handling Missing Values*

##### Before

In [1081]:
# Displaying a summary of the DataFrame, including:
# - Number of entries (rows)
# - Column names and their data types
# - Number of non-null (non-missing) values per column
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99 entries, 0 to 99
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_id       99 non-null     int64  
 1   customer_name  98 non-null     object 
 2   product        99 non-null     object 
 3   quantity       73 non-null     float64
 4   unit_price     64 non-null     float64
 5   order_date     98 non-null     object 
 6   region         74 non-null     object 
dtypes: float64(2), int64(1), object(4)
memory usage: 6.2+ KB


##### Transformation

In [1082]:
# Drop rows where both unit_price and quantity are missing
df_full = df_full[~(df_full['unit_price'].isna() & df_full['quantity'].isna())]


In [1083]:
# Check the skewness of the 'quantity' column to understand its distribution
df_full['quantity'].skew()


0.07738528153496914

In [1084]:
# Fill missing values in 'quantity' with the mean, rounded to the nearest integer
df_full['quantity'] = df_full['quantity'].fillna(round(df_full['quantity'].mean()))

# Fill missing 'unit_price' with the median unit_price per product
df_full['unit_price'] = df_full.groupby('product')['unit_price'].transform(lambda x: x.fillna(x.median()))

# Fill customer_name with 'Unknown'
df_full['customer_name'] = df_full['customer_name'].fillna('Unknown')

# Fill region with its mode (most frequent value)
region_mode = df_full['region'].mode()[0]
df_full['region'] = df_full['region'].fillna(region_mode)

# Drop row where 'order_date' is missing
df_full = df_full[df_full['order_date'].notna()]


##### After

In [1085]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90 entries, 0 to 99
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_id       90 non-null     int64  
 1   customer_name  90 non-null     object 
 2   product        90 non-null     object 
 3   quantity       90 non-null     float64
 4   unit_price     90 non-null     float64
 5   order_date     90 non-null     object 
 6   region         90 non-null     object 
dtypes: float64(2), int64(1), object(4)
memory usage: 5.6+ KB


___
##### Transformation : Explanation

- **Removed rows where both `unit_price` and `quantity` were missing**  
  ➤ These are critical fields for sales analysis. If both are missing, the row provides no useful financial information.

- **Filled missing `quantity` with its mean (rounded)**  
  ➤ Since `quantity` had low skewness, the mean is a fair estimate. Rounding maintains the expected integer format.

- **Filled missing `unit_price` using the median per product**  
  ➤ Median is robust against outliers and ensures more accurate pricing per product category.

- **Filled missing `customer_name` with `"Unknown"`**  
  ➤ Helps retain the record without mislabeling or dropping potentially useful sales data.

- **Filled missing `region` with the most frequent value (mode)**  
  ➤ This ensures geographic coverage is maintained while relying on the most common, likely region.

- **Dropped row with missing `order_date`**  
  ➤ Since only one record was affected and dates are key for time-based analysis, dropping was the safest option.

✅ **Why this transformation is important:**  
These steps improve **data quality**, reduce **bias or errors**, and ensure that the dataset supports **reliable insights** in downstream analysis and visualization.
___


#### **2. Structural Transformation** 🛠️

##### Before

In [1086]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90 entries, 0 to 99
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_id       90 non-null     int64  
 1   customer_name  90 non-null     object 
 2   product        90 non-null     object 
 3   quantity       90 non-null     float64
 4   unit_price     90 non-null     float64
 5   order_date     90 non-null     object 
 6   region         90 non-null     object 
dtypes: float64(2), int64(1), object(4)
memory usage: 5.6+ KB


##### Transformation

In [1087]:
# Convert 'order_id' to string if it's used as an identifier, not for calculation
df_full['order_id'] = df_full['order_id'].astype(str)

# Convert 'order_date' to datetime format for proper time-based analysis
df_full['order_date'] = pd.to_datetime(df_full['order_date'])

# Convert 'quantity' to integer type, as it represents whole items sold
df_full['quantity'] = df_full['quantity'].astype(int)

# Convert 'product' and 'region' to categorical types for memory and efficiency benefits
df_full['product'] = df_full['product'].astype('category')
df_full['region'] = df_full['region'].astype('category')


##### After

In [1088]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90 entries, 0 to 99
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       90 non-null     object        
 1   customer_name  90 non-null     object        
 2   product        90 non-null     category      
 3   quantity       90 non-null     int32         
 4   unit_price     90 non-null     float64       
 5   order_date     90 non-null     datetime64[ns]
 6   region         90 non-null     category      
dtypes: category(2), datetime64[ns](1), float64(1), int32(1), object(2)
memory usage: 4.4+ KB


___
##### Transformation: Explanation

-  **order_id**: Converted to string since it serves as a unique identifier, not for numeric operations.
-  **order_date**: Converted to `datetime` format to enable time-based filtering, sorting, and analysis.
-  **quantity**: Converted to integer type, as it represents whole items sold.
-  **product & region**: Converted to `category` type to reduce memory usage and improve performance when grouping or filtering.

These changes help maintain data integrity, improve efficiency, and prepare the data for further analysis or modeling.
___


#### **3. ✨ Enrichment**

##### Before

In [1089]:
# Print the list of column names in the DataFrame
print(df_full.columns.tolist())


['order_id', 'customer_name', 'product', 'quantity', 'unit_price', 'order_date', 'region']


##### Transformation

In [1090]:
# 💰 Add total_price column: total = quantity × unit_price
df_full['total_price'] = df_full['quantity'] * df_full['unit_price']

##### After

In [1091]:
#  Print the list of column names in the DataFrame
print(df_full.columns.tolist())


['order_id', 'customer_name', 'product', 'quantity', 'unit_price', 'order_date', 'region', 'total_price']


___
##### Transformation: Explanation

- 💰 **Add `total_price` column:**  
  Calculated by multiplying `quantity` and `unit_price`.

- **Why?**  
  This enriches the dataset with a crucial metric — **total revenue per order** — which supports:
  - Sales performance tracking
  - Revenue-based analytics
  - Grouped aggregations by region or product
___


#### **4. 🧩 Categorization**

##### Before

In [1092]:
df_full.head()

,order_id,customer_name,product,quantity,unit_price,order_date,region,total_price
0,1,Diana,Tablet,2,500.0,2024-01-20,South,1000.0
2,3,Charlie,Laptop,2,250.0,2024-01-08,South,500.0
3,4,Eve,Laptop,2,750.0,2024-01-07,West,1500.0
4,5,Eve,Tablet,3,500.0,2024-03-07,South,1500.0
6,7,Charlie,Monitor,2,750.0,2024-02-02,West,1500.0


##### Transformation

In [1093]:
# 🏷️ Categorize 'unit_price' into price segments based on percentiles

# Get the 25th and 75th percentiles
q1 = df_full['unit_price'].quantile(0.25)
q3 = df_full['unit_price'].quantile(0.75)

# Define a function to classify unit prices
def categorize_price(price):
    if price <= q1:
        return 'Low-cost'
    elif price <= q3:
        return 'Mid-range'
    else:
        return 'Premium'

# Apply the function to create a new column
df_full['price_category'] = df_full['unit_price'].apply(categorize_price)


##### After

In [1094]:
df_full.head()

,order_id,customer_name,product,quantity,unit_price,order_date,region,total_price,price_category
0,1,Diana,Tablet,2,500.0,2024-01-20,South,1000.0,Mid-range
2,3,Charlie,Laptop,2,250.0,2024-01-08,South,500.0,Low-cost
3,4,Eve,Laptop,2,750.0,2024-01-07,West,1500.0,Premium
4,5,Eve,Tablet,3,500.0,2024-03-07,South,1500.0,Mid-range
6,7,Charlie,Monitor,2,750.0,2024-02-02,West,1500.0,Premium


___
##### Transformation: Explanation

To enhance analysis and group similar items,  the `unit_price` was categorized column into three segments and added as new column `price_category` :

- **Low-cost**: Prices from the minimum up to the 25th percentile  
- **Mid-range**: Prices between the 25th and 75th percentiles  
- **Premium**: Prices above the 75th percentile

🔍 This segmentation helps in pricing analysis, customer segmentation, and product strategy.
___


In [1095]:
# Export the df_full DataFrame to a CSV file named ' transformed_full.csv' without including the index column
df_full.to_csv(" transformed_full.csv", index=False)

### **Incremental Data**

#### 🧹✨ **1. Cleaning**

In [1096]:
# Reading the CSV file named 'raw_data.csv' into a DataFrame called df_full
df_incremental = pd.read_csv("incremental_data.csv")

#### *Handling Missing Values*

##### Before

In [1097]:
# Displaying a summary of the DataFrame, including:
# - Number of entries (rows)
# - Column names and their data types
# - Number of non-null (non-missing) values per column
df_incremental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_id       10 non-null     int64  
 1   customer_name  4 non-null      object 
 2   product        10 non-null     object 
 3   quantity       6 non-null      float64
 4   unit_price     10 non-null     int64  
 5   order_date     10 non-null     object 
 6   region         8 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 692.0+ bytes


##### Transformation

In [1098]:
# Check the skewness of the 'quantity' column to understand its distribution
df_incremental['quantity'].skew()


0.0

In [1099]:
# Fill missing values in 'quantity' with the mean, rounded to the nearest integer
df_incremental['quantity'] =df_incremental['quantity'].fillna(round(df_incremental['quantity'].mean()))

# Fill customer_name with 'Unknown'
df_incremental['customer_name'] =df_incremental['customer_name'].fillna('Unknown')

# Fill region with its mode (most frequent value)
region_mode =df_incremental['region'].mode()[0]
df_incremental['region'] =df_incremental['region'].fillna(region_mode)


##### After

In [1100]:
df_incremental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_id       10 non-null     int64  
 1   customer_name  10 non-null     object 
 2   product        10 non-null     object 
 3   quantity       10 non-null     float64
 4   unit_price     10 non-null     int64  
 5   order_date     10 non-null     object 
 6   region         10 non-null     object 
dtypes: float64(1), int64(2), object(4)
memory usage: 692.0+ bytes


___
##### Transformation : Explanation

- **Filled missing `quantity` with its mean (rounded)**  
  ➤ Since `quantity` had zero skewness, the mean is a fair estimate. Rounding maintains the expected integer format.

- **Filled missing `customer_name` with `"Unknown"`**  
  ➤ Helps retain the record without mislabeling or dropping potentially useful sales data.

- **Filled missing `region` with the most frequent value (mode)**  
  ➤ This ensures geographic coverage is maintained while relying on the most common, likely region.

✅ **Why this transformation is important:**  
These steps improve **data quality**, reduce **bias or errors**, and ensure that the dataset supports **reliable insights** in downstream analysis and visualization.
___


#### **2. Structural Transformation** 🛠️

##### Before

In [1101]:
df_incremental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_id       10 non-null     int64  
 1   customer_name  10 non-null     object 
 2   product        10 non-null     object 
 3   quantity       10 non-null     float64
 4   unit_price     10 non-null     int64  
 5   order_date     10 non-null     object 
 6   region         10 non-null     object 
dtypes: float64(1), int64(2), object(4)
memory usage: 692.0+ bytes


##### Transformation

In [1102]:
# Convert 'order_id' to string if it's used as an identifier, not for calculation
df_incremental['order_id'] = df_incremental['order_id'].astype(str)

# Convert 'order_date' to datetime format for proper time-based analysis
df_incremental['order_date'] = pd.to_datetime(df_incremental['order_date'])

# Convert 'quantity' to integer type, as it represents whole items sold
df_incremental['quantity'] = df_incremental['quantity'].astype(int)

# Convert 'quantity' to integer type, as it represents whole items sold
df_incremental['unit_price'] = df_incremental['unit_price'].astype(float)

# Convert 'product' and 'region' to categorical types for memory and efficiency benefits
df_incremental['product'] = df_incremental['product'].astype('category')
df_incremental['region'] = df_incremental['region'].astype('category')

##### After

In [1103]:
df_incremental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       10 non-null     object        
 1   customer_name  10 non-null     object        
 2   product        10 non-null     category      
 3   quantity       10 non-null     int32         
 4   unit_price     10 non-null     float64       
 5   order_date     10 non-null     datetime64[ns]
 6   region         10 non-null     category      
dtypes: category(2), datetime64[ns](1), float64(1), int32(1), object(2)
memory usage: 768.0+ bytes


___
##### Transformation: Explanation

-  **order_id**: Converted to string since it serves as a unique identifier, not for numeric operations.
-  **order_date**: Converted to `datetime` format to enable time-based filtering, sorting, and analysis.
-  **quantity**: Converted to integer type, as it represents whole items sold.
-  **product & region**: Converted to `category` type to reduce memory usage and improve performance when grouping or filtering.
-  **price**: Converted to float type to accommodate decimal prices.

These changes help maintain data integrity, improve efficiency, and prepare the data for further analysis or modeling.
___


#### **3. ✨ Enrichment**

##### Before

In [1104]:
# Print the list of column names in the DataFrame
print(df_incremental.columns.tolist())

['order_id', 'customer_name', 'product', 'quantity', 'unit_price', 'order_date', 'region']


##### Transformation

In [1105]:
# 💰 Add total_price column: total = quantity × unit_price
df_incremental['total_price'] = df_incremental['quantity'] * df_incremental['unit_price']

##### After

In [1106]:
#  Print the list of column names in the DataFrame
print(df_incremental.columns.tolist())

['order_id', 'customer_name', 'product', 'quantity', 'unit_price', 'order_date', 'region', 'total_price']


___
##### Transformation: Explanation

- 💰 **Add `total_price` column:**  
  Calculated by multiplying `quantity` and `unit_price`.

- **Why?**  
  This enriches the dataset with a crucial metric — **total revenue per order** — which supports:
  - Sales performance tracking
  - Revenue-based analytics
  - Grouped aggregations by region or product
___


#### **4. 🧩 Categorization**

##### Before

In [1107]:
df_incremental.head()

,order_id,customer_name,product,quantity,unit_price,order_date,region,total_price
0,101,Alice,Laptop,2,900.0,2024-05-09,Central,1800.0
1,102,Unknown,Laptop,1,300.0,2024-05-07,Central,300.0
2,103,Unknown,Laptop,1,600.0,2024-05-04,Central,600.0
3,104,Unknown,Tablet,2,300.0,2024-05-26,Central,600.0
4,105,Heidi,Tablet,2,600.0,2024-05-21,North,1200.0


##### Transformation

In [1108]:
# 🏷️ Categorize 'unit_price' into price segments based on percentiles

# Get the 25th and 75th percentiles
q1 = df_incremental['unit_price'].quantile(0.25)
q3 = df_incremental['unit_price'].quantile(0.75)

# Define a function to classify unit prices
def categorize_price(price):
    if price <= q1:
        return 'Low-cost'
    elif price <= q3:
        return 'Mid-range'
    else:
        return 'Premium'

# Apply the function to create a new column
df_incremental['price_category'] = df_incremental['unit_price'].apply(categorize_price)


##### After

In [1109]:
df_incremental.head()

,order_id,customer_name,product,quantity,unit_price,order_date,region,total_price,price_category
0,101,Alice,Laptop,2,900.0,2024-05-09,Central,1800.0,Premium
1,102,Unknown,Laptop,1,300.0,2024-05-07,Central,300.0,Low-cost
2,103,Unknown,Laptop,1,600.0,2024-05-04,Central,600.0,Low-cost
3,104,Unknown,Tablet,2,300.0,2024-05-26,Central,600.0,Low-cost
4,105,Heidi,Tablet,2,600.0,2024-05-21,North,1200.0,Low-cost


In [1110]:
# Export the df_full DataFrame to a CSV file named ' transformed_full.csv' without including the index column
df_incremental.to_csv(" transformed_incremental.csv", index=False)

___
##### Transformation: Explanation

To enhance analysis and group similar items,  the `unit_price` was categorized column into three segments and added as new column `price_category` :

- **Low-cost**: Prices from the minimum up to the 25th percentile  
- **Mid-range**: Prices between the 25th and 75th percentiles  
- **Premium**: Prices above the 75th percentile

🔍 This segmentation helps in pricing analysis, customer segmentation, and product strategy.
___
